This notebook shows every step of our query expansion algorithm. We will also evaluate how different parts performs both on training data ans unseen test data

In [1]:
from collections import Counter
from string import punctuation
import spacy
import os
import numpy as np
import json
import elasticsearch
from elasticsearch import Elasticsearch
import utils
import spacy
import evaluator
from evaluator import train_topics, train_qrels, test_topics, test_qrels
from nltk.stem import PorterStemmer 

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [65]:
es = Elasticsearch(http_auth=(os.environ['ES_USER'], os.environ['ES_PWD']))

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
ps = PorterStemmer() 

In [6]:
def init_pos_vocab(X):
    pos_vocab = []
    for topic in X:
        for turn in topic['turn']:
            doc = nlp(turn['raw_utterance'])
            for token in doc:
                if token.pos_ not in pos_vocab:
                    pos_vocab.append(token.pos_)
    return pos_vocab

In [7]:
pos_vocab = init_pos_vocab(train_topics)
pos_vocab

['PRON',
 'AUX',
 'DET',
 'NOUN',
 'PART',
 'PUNCT',
 'ADJ',
 'VERB',
 'NUM',
 'ADP',
 'PROPN',
 'CCONJ',
 'ADV',
 'SPACE',
 'SCONJ']

In [8]:
len(pos_vocab)

15

### Vector representation of POS tags in text

In [9]:
def pos_parser(text, pos_vocab):
    doc = nlp(text)
    tags = []
    res = []
    for token in doc:
        if token.pos_ in pos_vocab:
            tags.append(token.pos_)
            res.append(token.text)
    return res, tags

In [10]:
t = train_topics[0]['turn'][0]['raw_utterance']

In [11]:
q = evaluator.analyzer(es, t, 'trec2019_stem')
q = ' '.join(q)
print(q)
txt, pos = pos_parser(q, pos_vocab)

what is a physician's assistant


In [12]:
def get_pos_counts(pos_list, pos_vocab):
        return [pos_list.count(i) for i in pos_vocab]

In [13]:
get_pos_counts(pos, pos_vocab)

[1, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## Generate POS vectors for all turns

In [14]:
def get_category(pos):
    num = sum([1 for tag in pos if tag in ['NOUN', 'PROPN']])
    if num > 2:
        return 1
    return 2

In [15]:
cat1 = {}
cat2 = {} # Need context
index = {}
for topic in train_topics:
    for turn in topic['turn']:
        qid = turn['qid']
        _qrels = utils.get_qrels(qid, train_qrels)
        q = ' '.join(evaluator.analyzer(es, turn['raw_utterance'], 'trec2019_stem'))
        index[qid] = {'text': q}
        if not _qrels: # Only include turns with labeled relevancy
            continue
        txt, pos = pos_parser(q, pos_vocab)
        pos_vec = get_pos_counts(pos, pos_vocab)
        cat = get_category(pos)
        if turn['number'] == 1 or cat==1:
            cat1[qid] = {'pos_vec': pos_vec}
        else:
            cat2[qid] = {'pos_vec': pos_vec}

In [16]:
get_category(pos)

2

Access POS vector from category 1

In [17]:
cat1['1_1']

{'pos_vec': [1, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

Access POS vector from category 2

In [18]:
cat2['1_2']

{'pos_vec': [1, 1, 1, 1, 1, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0]}

Access query from index

In [19]:
index['1_2']

{'text': 'what are the educational requirements required to become one'}

# Category 1
## Euclidean distance

In [20]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((np.array(x) - np.array(y)) ** 2))

In [21]:
cat1_vec = [i['pos_vec'] for _, i in cat1.items()]
cat2_vec = [i['pos_vec'] for _, i in cat2.items()]

In [22]:
def knn(pos_vec, cat, qid='None', k=1):
    dists = {}
    for key, v in cat.items():
        if key != qid:
            dist = euclidean_distance(v['pos_vec'], pos_vec)
            dists[key] = dist
    dists = {key: v for key, v in sorted(dists.items(), key=lambda item: item[1])}
    return {key:dists[key] for key in list(dists.keys())[:k]}

In [23]:
q = ' '.join(evaluator.analyzer(es, index['1_1']['text'], 'trec2019_stem'))
_, pos = pos_parser(q, pos_vocab)
nb = knn(get_pos_counts(pos, pos_vocab), cat1, '1_1', 3)
print(f'Original: {index["1_1"]["text"]}')
print("3 nearest neighbors")
for n in nb:
    print(index[n]["text"])

Original: what is a physician's assistant
3 nearest neighbors
what are the main breeds of goat
what was the neolithic revolution
what are the different types of macromolecules


We can see that the nearest neighbor queries are pretty similar to the original one

## Reverse engineer perfect query

In [24]:
def get_top_res(qid, qrels):
    """Only return those with relevancy above 1"""
    _qrels = utils.get_qrels(qid, qrels)
    if not _qrels:
        return None
    ground_truth = utils.get_ground_truth(_qrels)
    return {k: v for k, v in sorted(ground_truth.items(), key=lambda item: item[1], reverse=True) if v>0}

In [25]:
qid = '1_1'
top_res = get_top_res(qid, train_qrels)
top_res

{'MARCO_955948': 2,
 'MARCO_6203672': 2,
 'MARCO_955945': 2,
 'MARCO_6203671': 2,
 'MARCO_5692406': 1}

In [26]:
def get_keywords(text, pos_vocab, k=20):
    doc = nlp(text)
    tokens = [token.text for token in doc if token.pos_ in pos_vocab]
    tokens = Counter(tokens).most_common(k) # [(word, count), (word, count)..] descending order by count
    return tokens

def get_kw_stats(top_res, es, index, pos_vocab):
    res = {}
    kw_count = {}
    for doc_id, relevance in top_res.items():
        search_res = es.get(index=index, id=doc_id)
        doc = search_res['_source']['body']
        doc = ' '.join(evaluator.analyzer(es, doc, index))
        keywords = get_keywords(doc, pos_vocab)
        for kw in keywords:
            if kw[0] not in kw_count:
                kw_count[kw[0]] = 0
            kw_count[kw[0]] +=  kw[1] # 2 in relevancy more important
        res[doc_id] = {'doc': doc, 'keywords': keywords}
    return res, kw_count

In [27]:
data, kw_count = get_kw_stats(top_res, es, 'trec2019_stem', pos_vocab)

In [28]:
data['MARCO_955948']

{'doc': 'physician assistants work under the supervision of a physician or surgeon however their specific duties and the extent to which they must be supervised differ from state to state physician assistants work in all areas of medicine including primary care and family medicine emergency medicine and psychiatry.the work of physician assistants depends in large part on their specialty and what their supervising physician needs them to do.for example a physician assistant working in surgery may close incisions and provide care before and after the operation.he work of physician assistants depends in large part on their specialty and what their supervising physician needs them to do for example a physician assistant working in surgery may close incisions and provide care before and after the operation',
 'keywords': [('physician', 9),
  ('and', 9),
  ('their', 5),
  ('in', 5),
  ('assistants', 4),
  ('work', 4),
  ('the', 4),
  ('of', 4),
  ('to', 4),
  ('a', 3),
  ('medicine', 3),
  (

In [29]:
topk_kw = {key:kw_count[key] for key in list(kw_count.keys())}
print(topk_kw)

{'physician': 34, 'and': 32, 'their': 17, 'in': 14, 'assistants': 13, 'work': 7, 'the': 11, 'of': 11, 'to': 8, 'a': 16, 'medicine': 7, 'care': 6, 'state': 2, 'depends': 4, 'large': 4, 'part': 4, 'on': 8, 'specialty': 4, 'what': 6, 'supervising': 4, 'needs': 2, 'them': 2, 'do': 2, 'provide': 4, 'also': 2, 'known': 2, 'as': 3, 'pas': 1, 'practice': 1, 'team': 1, 'diagnostic': 1, 'therapeutic': 1, 'preventive': 1, 'healthcare': 1, 'services': 1, 'delegated': 1, 'by': 1, 'pa': 5, 'assistant': 5, 'is': 4, 'medical': 3, 'c': 3, 'md': 2, 'doctor': 2, 'an': 2, "'s": 2, 'are': 2, 'fully': 2, 'qualified': 2, 'physicians': 2, 'graduated': 2, 'from': 2, 'accredited': 2}


In [30]:
index[qid]['text']

"what is a physician's assistant"

In [31]:
def get_base_score(q, qid, qrels, es, index, size=100):
    search_results, system_ranking, ground_truth = evaluator.get_search_data(q,
                                                               qid,
                                                               qrels,
                                                               es,
                                                               index,
                                                               size
                                                              )
    return utils.ndcg(system_ranking, ground_truth, k=3)

In [32]:
base_score = get_base_score(index[qid]['text'], qid, train_qrels, es, 'trec2019_stem')
print(f'Original score was: {base_score}\n')
last_query = []
top_query = {'score': 0, 'q': None}
for s in range(20, 1, -1):
    query = [i for i in topk_kw if i not in nlp.Defaults.stop_words][:s] # Top s keywords
    if query == last_query:
        continue
    last_query = query
    print(query)
    search_results, system_ranking, ground_truth = evaluator.get_search_data(query,
                                                                   qid,
                                                                   train_qrels,
                                                                   es,
                                                                   'trec2019_stem',
                                                                   100
                                                                  )
    if system_ranking is None:
        continue

    score = utils.ndcg(system_ranking, ground_truth, k=3)
    if score >= top_query['score']:
        top_query = {'score': score, 'q': query}
    print(s, " score ", score)
print(f'\nTop query: {top_query}')

Original score was: 0.0

['physician', 'assistants', 'work', 'medicine', 'care', 'state', 'depends', 'large', 'specialty', 'supervising', 'needs', 'provide', 'known', 'pas', 'practice', 'team', 'diagnostic', 'therapeutic', 'preventive', 'healthcare']
20  score  0.6199062332840657
['physician', 'assistants', 'work', 'medicine', 'care', 'state', 'depends', 'large', 'specialty', 'supervising', 'needs', 'provide', 'known', 'pas', 'practice', 'team', 'diagnostic', 'therapeutic', 'preventive']
19  score  0.6199062332840657
['physician', 'assistants', 'work', 'medicine', 'care', 'state', 'depends', 'large', 'specialty', 'supervising', 'needs', 'provide', 'known', 'pas', 'practice', 'team', 'diagnostic', 'therapeutic']
18  score  0.7601875334318686
['physician', 'assistants', 'work', 'medicine', 'care', 'state', 'depends', 'large', 'specialty', 'supervising', 'needs', 'provide', 'known', 'pas', 'practice', 'team', 'diagnostic']
17  score  1.0
['physician', 'assistants', 'work', 'medicine', 'ca

#### Analyze which POS tags were used in the "perfect" query

In [33]:
q = ' '.join([i for i in top_query['q']])
txt, pos =  pos_parser(q, pos_vocab)
print(pos)
print("Unique: ", set(pos))

['NOUN', 'NOUN', 'PROPN', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'VERB']
Unique:  {'ADJ', 'NOUN', 'VERB', 'PROPN'}


In [34]:
get_pos_counts(pos, pos_vocab)

[0, 0, 0, 8, 0, 0, 1, 3, 0, 0, 1, 0, 0, 0, 0]

We can clearly see that some pos tags are more important than others, especially NOUN tags. We can use this information to filter out new queries. Reverse engineer all queries in category 1 and store the results in the cat1 dictionary

In [35]:
def get_perfect_query(qid, qrels, es, index, index_name, pos_vocab, size=100):
    base_score = get_base_score(index[qid]['text'], qid, qrels, es, index_name)
    top_res = get_top_res(qid, qrels)        
    _, kw_count = get_kw_stats(top_res, es, index_name, pos_vocab)
    topk_kw = {key:kw_count[key] for key in list(kw_count.keys())}
    last_query = []
    top_query = {'score': 0, 'base_score': base_score ,'q': None}
    for s in range(20, 1, -1):
        query = [i for i in topk_kw if i not in nlp.Defaults.stop_words][:s] # Top s keywords
        if query == last_query:
            continue
        query = evaluator.analyzer(es, ' '.join(query), 'trec2019_stem')
        last_query = query
        search_results, system_ranking, ground_truth = evaluator.get_search_data(query,
                                                                       qid,
                                                                       qrels,
                                                                       es,
                                                                       index_name,
                                                                       size
                                                                      )
        if system_ranking is None:
            continue

        score = utils.ndcg(system_ranking, ground_truth, k=3)
        if score >= top_query['score']:
            top_query = {'score': score, 'base_score': base_score, 'q': query}
            
    q = ' '.join([i for i in top_query['q']])
    txt, pos =  pos_parser(q, pos_vocab)
    pos_vec = get_pos_counts(pos, pos_vocab)
    top_query['pos_vec'] = pos_vec
    top_query['pos'] = set(pos)
    return top_query

In [36]:
get_perfect_query('1_1', train_qrels, es, index, 'trec2019_stem', pos_vocab)

{'score': 1.0,
 'base_score': 0.0,
 'q': ['physician',
  'assistants',
  'work',
  'medicine',
  'care',
  'state',
  'depends',
  'large',
  'specialty',
  'supervising',
  'needs',
  'provide',
  'known'],
 'pos_vec': [0, 0, 0, 8, 0, 0, 1, 3, 0, 0, 1, 0, 0, 0, 0],
 'pos': {'ADJ', 'NOUN', 'PROPN', 'VERB'}}

In [37]:
for qid in cat1:
    perfect_query = get_perfect_query(qid, train_qrels, es, index, 'trec2019_stem', pos_vocab)
    print(perfect_query,"\n")
    cat1[qid]['perfect_query'] = perfect_query

{'score': 1.0, 'base_score': 0.0, 'q': ['physician', 'assistants', 'work', 'medicine', 'care', 'state', 'depends', 'large', 'specialty', 'supervising', 'needs', 'provide', 'known'], 'pos_vec': [0, 0, 0, 8, 0, 0, 1, 3, 0, 0, 1, 0, 0, 0, 0], 'pos': {'ADJ', 'NOUN', 'VERB', 'PROPN'}} 

{'score': 1.0, 'base_score': 0.0, 'q': ['physician', 'starting', 'salary', 'assistantâ', 's', 'companies'], 'pos_vec': [0, 0, 0, 4, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], 'pos': {'NOUN', 'VERB', 'PROPN'}} 

{'score': 1.0, 'base_score': 1.0, 'q': ['school', 'nursing', 'high'], 'pos_vec': [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'pos': {'ADV', 'NOUN'}} 

{'score': 0.82623465712856, 'base_score': 0.0, 'q': ['average', 'salary', 'physician', '2014', 'assistant', 'pay', 'vs', 'best', 'jobs', 'assistants', '97,280', 'nurse', 'practitioners', '2013', 'doctors', 'paid', 'fact', 'physicianâ'], 'pos_vec': [0, 0, 0, 8, 0, 0, 3, 2, 3, 1, 1, 0, 0, 0, 0], 'pos': {'PROPN', 'NUM', 'ADP', 'VERB', 'ADJ', 'NOUN'}} 

{'score':

### Filter query based on nearest neighbor perfect query
For each query that falls in category 1, find the nearests neighbor in the cat1 dictionary and look at its perfect query. We can now filter any new query based on this

In [38]:
qid = '1_1'
q = ' '.join(evaluator.analyzer(es, index['1_1']['text'], 'trec2019_stem'))
_, pos = pos_parser(q, pos_vocab)
nb = knn(get_pos_counts(pos, pos_vocab), cat1, qid, 3)
pos_filter = []
for n in nb: # n=qid
    pos_filter = pos_filter + list(cat1[n]['perfect_query']['pos'])
pos_filter = set(pos_filter)
pos_filter

{'ADJ', 'NOUN', 'PROPN', 'VERB'}

In [39]:
q = ' '.join(evaluator.analyzer(es, index[qid]['text'], 'trec2019_stem'))
parsed_query, pos = pos_parser(q, pos_filter)
print(f'Original: {q}\nParsed: {parsed_query}')

Original: what is a physician's assistant
Parsed: ['physician', 'assistant']


Our algorithm thinks "what is a" is unuseful data for this query

In [40]:
sres, srank, gt = evaluator.get_search_data(parsed_query, qid, train_qrels, es, 'trec2019_stem')
score = utils.ndcg(srank, gt, k=3)
base_score = cat1[qid]['perfect_query']['base_score']
print(f'Original score: {base_score}\nParsed score: {score}')

Original score: 0.0
Parsed score: 0.0


We still get 0 in score even though the query seems to search for the key part of the users intuition. If we look at the perfect query above for this one, we can see that the two best keywords in fact is ['physician', 'assistant'], but it needs some additional data in order to receive relevant ones. We should therefore try to expand our filtered query in some way.

### Expand filtered query
The strategy here is to fetch top 100 documents from a base retrieval for the query. We will then compute the top keywords from these documents based on our pos filter, and then expand the parsed query with the most important keywords 

In [41]:
search_results = es.search(index='trec2019_stem', q=q, _source=False, size=100)['hits']['hits']
base_ret = {i["_id"]:1 for i in search_results}
_, kw_count = get_kw_stats(base_ret, es, 'trec2019_stem', pos_filter)

In [42]:
def expand_query(kw, query, k):
    # We need to stem the words so we don't add similar words
    # Ex: assistants will not be added to query [physician, assistant]
    _kw = [ps.stem(w) for w in kw]
    _q = [ps.stem(w) for w in query]
    expanded_q = query.copy()
    for stemmed_term,  term in zip(_kw, kw):
        if stemmed_term not in _q:
            expanded_q.append(term)
    return expanded_q[:k]

In [43]:
expanded_q = expand_query(kw_count, parsed_query, len(parsed_query)+3)
print(expanded_q)

['physician', 'assistant', 'healthcare', 'professionals', 'educated']


In [44]:
sres, srank, gt = evaluator.get_search_data(expanded_q, qid, train_qrels, es, 'trec2019_stem')
score = utils.ndcg(srank, gt, k=3)
base_score = cat1[qid]['perfect_query']['base_score']
print(f'Original score: {base_score}\nParsed score: {score}')

Original score: 0.0
Parsed score: 0.0


The expansion technique did not work for this example.
### Compare expansion vs no expansion

In [45]:
parsed_scores = []
base_scores = []
expanded_scores = {k:[] for k in range(1,21)}
for qid in cat1:
    _q = ' '.join(evaluator.analyzer(es, index[qid]['text'], 'trec2019_stem'))
    _, pos = pos_parser(_q, pos_vocab)
    nb = knn(get_pos_counts(pos, pos_vocab), cat1, qid, 3)
    pos_filter = []
    for n in nb: # n=qid
        pos_filter = pos_filter + list(cat1[n]['perfect_query']['pos'])
    pos_filter = set(pos_filter)
    q = ' '.join(evaluator.analyzer(es, index[qid]['text'], 'trec2019_stem'))
    parsed_query, pos = pos_parser(q, pos_filter)
    
    sres, srank, gt = evaluator.get_search_data(parsed_query, qid, train_qrels, es, 'trec2019_stem')
    score = utils.ndcg(srank, gt, k=3)
    base_score = cat1[qid]['perfect_query']['base_score']
    # Expand query (test with expanding size k=1,..,20)
    search_results = es.search(index='trec2019_stem', q=q, _source=False, size=100)['hits']['hits']
    base_ret = {i["_id"]:1 for i in search_results}
    _, kw_count = get_kw_stats(base_ret, es, 'trec2019_stem', pos_filter)
    expanded_q = {k: expand_query(kw_count, parsed_query, len(parsed_query)+k) for k in range(1,21)}
    for k, v in expanded_q.items():
        sres, srank, gt = evaluator.get_search_data(v, qid, train_qrels, es, 'trec2019_stem')
        expanded_score = utils.ndcg(srank, gt, k=3)
        expanded_scores[k].append(expanded_score)
    
    parsed_scores.append(score)
    base_scores.append(base_score)

In [46]:
avg_parsed = sum(parsed_scores)/len(parsed_scores)
avg_base = sum(base_scores)/len(base_scores)
print(f'Base scores: {base_scores}\nAverage: {avg_base}')
print(f'\nParsed scores: {parsed_scores}\nAverage: {avg_parsed}\n')
for k, v in expanded_scores.items():
    avg_expanded = sum(v)/len(v)
    print(f'Average expanded score for k={k} : {avg_expanded}')

Base scores: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.42985934992609853, 0.0, 0.23981246656813146, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3800937667159343, 0.0]
Average: 0.10248827916050822

Parsed scores: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.6199062332840657, 0.23981246656813146, 0.0, 0.7601875334318686, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6199062332840657, 0.3800937667159343, 0.3800937667159343, 0.0]
Average: 0.2

Average expanded score for k=1 : 0.10350703250369508
Average expanded score for k=2 : 0.09751172083949179
Average expanded score for k=3 : 0.08552109751108522
Average expanded score for k=4 : 0.07850703250369508
Average expanded score for k=5 : 0.06300937667159343
Average expanded score for k=6 : 0.06300937667159343
Average expanded score for k=7 : 0.0665164091752885
Average expanded score for k=8 : 0.0665164091752885
Average expanded score for k=9 : 0.0665164091752885
Average expanded score for k=10 : 0.0665164091752885
Average expanded score for k=11 : 0.0665164091752885
Average 

The query expansion technique does not improve the score for the examples we have here. We can see that the score declines as we expand the parsed query with more terms. The parsed term technique did however almost double the original score!

### Test with a new unknown query 

In [47]:
original_q = test_topics[58-31]['turn'][0]['raw_utterance']
qid =  test_topics[58-31]['turn'][0]['qid']
q = ' '.join(evaluator.analyzer(es, original_q, 'trec2019_stem'))
_, pos = pos_parser(q, pos_vocab)
nb = knn(get_pos_counts(pos, pos_vocab), cat1, k=3)
print(f'Original: {original_q}')
print("3 nearest neighbors")
for n in nb:
    print(index[n]["text"])

Original: What is a real-time database?
3 nearest neighbors
what are the main breeds of goat
what was the neolithic revolution
what are the different types of macromolecules


In [48]:
pos_filter = []
for n in nb: # n=qid
    pos_filter = pos_filter + list(cat1[n]['perfect_query']['pos'])
pos_filter = set(pos_filter)
print(pos_filter)

{'ADJ', 'NOUN', 'VERB', 'PROPN'}


The filter seems makes sense since the query asks for information based on numerical data. Lets see how the algorithm parses the original query

In [49]:
parsed_query, pos = pos_parser(q, pos_filter)
print(f'Original: {original_q}')
print(parsed_query)

Original: What is a real-time database?
['real', 'time', 'database']


In [50]:
sres, srank, gt = evaluator.get_search_data(parsed_query, qid, test_qrels, es, 'trec2019_stem')
score = utils.ndcg(srank, gt, k=3)
sres, srank, gt = evaluator.get_search_data(q, qid, test_qrels, es, 'trec2019_stem')
base_score = utils.ndcg(srank, gt, k=3)
print(f'Base score: {base_score}\nParsed score: {score}')

Base score: 0.4649296749630493
Parsed score: 0.5248827916050821


Our algorithm was able to clean the original query and improve the base score for this example!

### Check if the algorithm for category 1 improves the score

In [51]:
base_scores = []
parsed_scores = []
total = 0
for topic in test_topics:
    for turn in topic['turn']:
        qid = turn['qid']
        _qrels = utils.get_qrels(qid, test_qrels)
        if not _qrels: # Only include turns with labeled relevancy
            continue
        total += 1
        original_query = turn['raw_utterance']
        q = ' '.join(evaluator.analyzer(es, original_query, 'trec2019_stem'))
        txt, pos = pos_parser(q, pos_vocab)
        pos_vec = get_pos_counts(pos, pos_vocab)
        cat = get_category(pos)
        if turn['number'] == 1 or cat==1:
            nb = knn(get_pos_counts(pos, pos_vocab), cat1, k=3)
            pos_filter = []
            for n in nb: # n=qid
                pos_filter = pos_filter + list(cat1[n]['perfect_query']['pos'])
            pos_filter = set(pos_filter)
            parsed_query, pos = pos_parser(q, pos_filter)
            
            sres, srank, gt = evaluator.get_search_data(parsed_query, qid, test_qrels, es, 'trec2019_stem')
            if srank is None:
                continue
            score = utils.ndcg(srank, gt, k=3)
            sres, srank, gt = evaluator.get_search_data(q, qid, test_qrels, es, 'trec2019_stem')
            if srank is None:
                continue
            base_score = utils.ndcg(srank, gt, k=3)
            print(f'{original_query}({base_score}) -> {parsed_query}({score})')
            base_scores.append(base_score)
            parsed_scores.append(score)

What is throat cancer?(1.0) -> ['throat', 'cancer'](1.0)
What are the different types of sharks?(0.20663541109468855) -> ['different', 'types', 'sharks'](0.20663541109468855)
Tell me about the Neverending Story film.(0.06377673039867192) -> ['tell', 'neverending', 'story', 'film'](0.06377673039867192)
Tell me about the Bronze Age collapse.(0.09502344167898358) -> ['tell', 'bronze', 'age', 'collapse'](0.21492967496304927)
What other factors led to a breakdown of trade?(0.0) -> ['other', 'factors', 'led', 'breakdown', 'trade'](0.0)
What was the Stanford Experiment?(0.4400468833579672) -> ['stanford', 'experiment'](0.3450234416789836)
What were the similarities and differences between the studies?(0.0) -> ['similarities', 'differences', 'between', 'studies'](0.0)
What are the origins of popular music? (0.0) -> ['origins', 'popular', 'music'](0.0)
How was Netflix started? (0.0) -> ['netflix', 'started'](0.0)
When did Netflix shift from DVDs to a streaming service?(0.20216745220099608) -> [

In [52]:
avg_base = sum(base_scores)/len(base_scores)
avg_parse = sum(parsed_scores)/len(parsed_scores)
print(f'{len(parsed_scores)} of {total} fell into category 2')
print(f'Average base score: {avg_base}\nAverage parsed score: {avg_parse}')

39 of 173 fell into category 2
Average base score: 0.25741431796779657
Average parsed score: 0.2757608276601204


The algorithm improved the original score by 0.0183

# Category 2
Most of the items could levearge information from the previous context of the conversation. We will try to do this in the algorithm for category 2.
## Reverse engineer perfect queries for category 2
Store results in the cat2 dictionary. We will use pos vectors for perfect queries in the algorithm later on 

In [53]:
for qid in cat2:
    perfect_query = get_perfect_query(qid, train_qrels, es, index, 'trec2019_stem', pos_vocab)
    q = ' '.join(perfect_query['q'])
    bs = perfect_query['base_score']
    s = perfect_query['score']
    print(f'{q} => base: {bs} => score: {s}')
    cat2[qid]['perfect_query'] = perfect_query

pa physician assistant educational requirements order certified graduate program accredited assistants required orthopedic nurse practitioner opa => base: 0.0 => score: 0.8983537904952533
resident tuition cost program => base: 0.0 => score: 0.7601875334318686
physician salary average assistant assistantâ => base: 0.0 => score: 1.0
average salary nurse practitioners physician assistants hour 2011 => base: 0.0 => score: 1.0
assistant physician medical supervision => base: 0.0 => score: 1.0
nurse practitioner years => base: 0.0 => score: 0.3800937667159343
goat boer dutch => base: 0.6199062332840657 => score: 1.0
goat meat boer highly breeds goats generally productive breed originated south s considered slow maturing good trait => base: 0.2754115523761867 => score: 1.0
goat dairy meat goats breed spanish good fiber breeds angora generally milk production profitable like small => base: 0.0 => score: 0.7043638493171503
goat boer dutch => base: 1.0 => score: 1.0
pygmy goat called classes ped

whales whale seen coast summer san year round california february watching new zealand kaikoura hauraki gulf seattle orca juan incredible => base: 0.0 => score: 1.0
whales whale seen coast summer => base: 0.0 => score: 1.0
captivity killer whales practice => base: 0.0 => score: 0.5508231047523734
diet need balanced high people sugar => base: 0.0 => score: 1.0
vitamins dietary recommendations beneficial men women womenâ s bodies different needs diet supplements healthy vitamin nutrients health best food balanced => base: 0.0 => score: 0.82623465712856
acid fat ice cream high list things dairy downside saturated want lose weight foods avoid pregnancy harm => base: 0.0 => score: 0.6229422381190666
fiber soluble â s => base: 0.0 => score: 0.43187871689428314
fiber consume increase => base: 0.0 => score: 1.0
windows virtual linux wine software machine run machines s t api applications application project available microsoft => base: 0.0 => score: 1.0
linux boot prefer windows reason people 

### Algorithm
The query rewriter algorithm expands the original parsed query with whatever tokens from previous history it thinks is most important. The tokens must have a pos-tag within the pos_filter that gets created by looking at the pos-tags of perfect queries from 3 nearest neighbors (most similar query from cat2). The algorithm will expand the original query by every possible combination of tokens until it reaches 25k combinations. It will then only add NOUN, PROPN and ADV tokens. It will compute the euclidean distance between the pos vector of each combination and the pos vector of the perfect query. All queries with maximum 0.5 distance compared to the top score will be filtered out. We then compute the average distance of the remaining queries and returns the first one with average length (best scores among those with average length). We take the average length since expanded queries tend to be both way too short and way too long. 

In [54]:
def rewrite_query(q, q_pos, history, pos_filter, pos_vocab, pq):
    #print('Original query ', q)
    expanded_queries = [q.copy()]
    expanded_queries_pos = [q_pos]
    for h in history:
        txt, pos = pos_parser(h, pos_filter)
        for t, p in zip(txt, pos):
            if len(expanded_queries) > 25000:
                if p in ['NOUN', 'PROPN', 'ADV']:
                    tmp = q.copy() + [t]
                    tmp_p = q_pos.copy() + [p]
                    expanded_queries.append(tmp)
                    expanded_queries_pos.append(tmp_p)
            else:
                for ep, e in zip(expanded_queries_pos ,expanded_queries):
                    if t not in e:
                        tmp = e + [t]
                        tmp_p = ep + [p]
                        expanded_queries.append(tmp)
                        expanded_queries_pos.append(tmp_p)
    #print(f'{len(expanded_queries)} expanded')
    pos_vecs  = [get_pos_counts(p, pos_vocab) for p in expanded_queries_pos]
    dists = [euclidean_distance(pq['pos_vec'], pos_vec) for pos_vec in pos_vecs]
    top_q = expanded_queries[np.argmin(dists)]
    s = dists[np.argmin(dists)]
    dists = [d for d in dists if (s-d) < .5]
    avg_dists_l = sum([len(d) for d in expanded_queries])/len(expanded_queries)

    for i in range(len(dists)):
        l = len(expanded_queries[i])
        #if l == round(avg_dists_l):
        #    top_q = expanded_queries[i]
        #break
        if l > len(top_q) and (s-dists[i]) < .5:
            top_q = expanded_queries[i]
    top_p = expanded_queries_pos[np.argmin(dists)]

    #print("Top query: ", top_q, " pos ", top_p, " score ", dists[np.argmin(dists)])
    return top_q

### Run algorithm on an example

In [55]:
qid = '27_3'
q = ' '.join(evaluator.analyzer(es, index[qid]['text'], 'trec2019_stem'))
_, pos = pos_parser(q, pos_vocab)
nb = knn(get_pos_counts(pos, pos_vocab), cat2, qid, 3)
pos_filter = []
print(f'Original: {q}')
for n in nb: # n=qid
    print(index[n]['text'])
    pos_filter = pos_filter + list(cat2[n]['perfect_query']['pos'])
pos_filter = set(pos_filter)
pos_filter

Original: what are some specific recommendations for women
what is the best for fiber production
what breed is good for meat
are angora goats good for it


{'ADJ', 'ADV', 'NOUN', 'PROPN', 'SCONJ', 'VERB'}

Parse the query based on the computed pos_filter

In [56]:
q = ' '.join(evaluator.analyzer(es, index[qid]['text'], 'trec2019_stem'))
parsed_query, pos = pos_parser(q, pos_filter)
print(f'Original: {q}\nParsed: {parsed_query}')

Original: what are some specific recommendations for women
Parsed: ['specific', 'recommendations', 'women']


Compare parsed query with base query

In [57]:
sres, srank, gt = evaluator.get_search_data(parsed_query, qid, train_qrels, es, 'trec2019_stem')
score = utils.ndcg(srank, gt, k=3)
base_score = cat2[qid]['perfect_query']['base_score']
print(f'Original score: {base_score}\nParsed score: {score}')

Original score: 0.0
Parsed score: 0.0


Both of them got 0 in score. This makes sense since previous information in the conversation is needed in order to retrieve relevant data

In [58]:
# History data for this conversation
for topic in train_topics:
    history = []
    done = False
    for turn in topic['turn']:
        if turn['qid'] == qid:
            done = True
            break
        txt = index[turn['qid']]['text']
        _, pos = pos_parser(txt, pos_vocab)
        cat = get_category(pos)
        if cat == 3:
            #print("####Topic shift. clean history")
            history = [txt]
        else:
            history.append(txt)
    if done:
        break
history

['what comprises a balanced diet', 'is it the same for men as well as women']

Only consider the most similar query's perfect query data when expanding the query

In [59]:
pq = cat2[list(nb.keys())[0]]['perfect_query']
print(pq)

{'score': 1.0, 'base_score': 0.0, 'q': ['fiber', 'angora', 'goat', 'length', 'shades', 'know', 'fine', 'bred', 'goats', 'produce', 'cashmere', 'milk'], 'pos_vec': [0, 0, 0, 5, 0, 0, 2, 2, 0, 0, 3, 0, 0, 0, 0], 'pos': {'ADJ', 'NOUN', 'VERB', 'PROPN'}}


Run algorithm and print new generated query

In [60]:
new_q = rewrite_query(parsed_query, pos, history, pos_filter, pos_vocab, pq)
print(f'\nGenerated new query: {new_q}')


Generated new query: ['specific', 'recommendations', 'women', 'comprises', 'balanced', 'diet', 'same', 'men', 'as', 'well']


The query seems to have added some useful tokens based on the context. Lets evaluate it against the base query with NDCG 3,100 and 1000 scores

In [61]:
_, srank, gt = evaluator.get_search_data(index[qid]['text'], qid, train_qrels, es, 'trec2019_stem', size=1000)
print('Base query NDCG@1000: ', utils.ndcg(srank, gt, k=1000),
      'Base query NDCG@100: ', utils.ndcg(srank, gt, k=100),
      " Base query NDCG@3: ", utils.ndcg(srank, gt, k=3))

_, srank, gt = evaluator.get_search_data(new_q, qid, train_qrels, es, 'trec2019_stem', size=1000)
print('Base query NDCG@1000: ', round(utils.ndcg(srank, gt, k=1000),5),
      'New query NDCG@100: ', round(utils.ndcg(srank, gt, k=100),5),
      " New query NDCG@3: ", round(utils.ndcg(srank, gt, k=3),5))

Base query NDCG@1000:  0.0 Base query NDCG@100:  0.0  Base query NDCG@3:  0.0
Base query NDCG@1000:  0.42947 New query NDCG@100:  0.40035  New query NDCG@3:  0.34753


The new query did clearly improve the original search!

### Check if the algorithm for category 2 improves the score

In [62]:
def get_history(qid, topics):
    history = []
    for topic in topics:
        history = []
        done = False
        for turn in topic['turn']:
            if turn['qid'] == qid:
                done = True
                break
            txt = turn['raw_utterance']
            _, pos = pos_parser(txt, pos_vocab)
            cat = get_category(pos)
            if cat == 3:
                #print("####Topic shift. clean history")
                history = [txt]
            else:
                history.append(txt)
        if done:
            break
    return history
            
#get_history('27_3')
get_history('31_2', test_topics)

['What is throat cancer?']

In [63]:
base_scores = []
parsed_scores = []
expanded_scores = []
total = 0
for topic in test_topics:
    print('Topic: ', topic['number'])
    for turn in topic['turn']:
        qid = turn['qid']
        _qrels = utils.get_qrels(qid, test_qrels)
        if not _qrels: # Only include turns with labeled relevancy
            continue
        total += 1
        original_query = turn['raw_utterance']
        q = ' '.join(evaluator.analyzer(es, original_query, 'trec2019_stem'))
        txt, pos = pos_parser(q, pos_vocab)
        pos_vec = get_pos_counts(pos, pos_vocab)
        cat = get_category(pos)
        if turn['number'] != 1 and cat==2:
            nb = knn(get_pos_counts(pos, pos_vocab), cat2, qid, 3)
            pos_filter = []
            for n in nb: # n=qid
                pos_filter = pos_filter + list(cat2[n]['perfect_query']['pos'])
            pos_filter = set(pos_filter)
            parsed_query, pos = pos_parser(q, pos_filter)
            
            # Base score
            _, srank, gt = evaluator.get_search_data(q, qid, test_qrels, es, 'trec2019_stem')
            if srank is None:
                continue
            base_scores.append(utils.ndcg(srank, gt, k=3))
            
            # Parsed score
            _, srank, gt = evaluator.get_search_data(parsed_query, qid, test_qrels, es, 'trec2019_stem')
            if srank is None:
                continue
            parsed_scores.append(utils.ndcg(srank, gt, k=3))
            
            # Algorithm
            history = get_history(qid, test_topics)
            pq = cat2[list(nb.keys())[0]]['perfect_query']
            new_q = rewrite_query(parsed_query, pos, history, pos_filter, pos_vocab, pq)
            _, srank, gt = evaluator.get_search_data(new_q, qid, test_qrels, es, 'trec2019_stem')
            if srank is None:
                continue
            expanded_scores.append(utils.ndcg(srank, gt, k=3))

Topic:  31
Topic:  32
Topic:  33
Topic:  34
Topic:  35
Topic:  36
Topic:  37
Topic:  38
Topic:  39
Topic:  40
Topic:  41
Topic:  42
Topic:  43
Topic:  44
Topic:  45
Topic:  46
Topic:  47
Topic:  48
Topic:  49
Topic:  50
Topic:  51
Topic:  52
Topic:  53
Topic:  54
Topic:  55
Topic:  56
Topic:  57
Topic:  58
Topic:  59
Topic:  60
Topic:  61
Topic:  62
Topic:  63
Topic:  64
Topic:  65
Topic:  66
Topic:  67
Topic:  68
Topic:  69
Topic:  70
Topic:  71
Topic:  72
Topic:  73
Topic:  74
Topic:  75
Topic:  76
Topic:  77
Topic:  78
Topic:  79
Topic:  80


In [64]:
avg_base = sum(base_scores)/len(base_scores)
avg_parse = sum(parsed_scores)/len(parsed_scores)
avg_expanded = sum(expanded_scores)/len(expanded_scores)
print(f'{len(parsed_scores)} of {total} fell into category 2')
print(f'Average base score: {avg_base}\nAverage parsed score: {avg_parse}\nAverage expanded score: {avg_expanded}')

133 of 173 fell into category 2
Average base score: 0.09823090556943129
Average parsed score: 0.10958319759847135
Average expanded score: 0.18114566757827463


Our algorithm doubled the ndcg@3 score by expanding the query with tokens it thinks is useful! This is results from the test data, meaning all queries are unknown and we only compare it against similar queries from the training data and their perfect reverse engineered queries. We can further improve the score by reranking these results. 